In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, PolynomialFeatures, OneHotEncoder, PowerTransformer
from sklearn.linear_model import LinearRegression, ElasticNetCV, MultiTaskElasticNetCV, MultiTaskLassoCV, LarsCV, RidgeCV
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, NuSVR, SVR
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA
from sklearn.compose import TransformedTargetRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, RandomizedSearchCV

import plotly.graph_objs as go
import plotly.offline as offline

# def enable_plotly_in_cell():
#   import IPython
#   from plotly.offline import init_notebook_mode
#   display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
#   init_notebook_mode(connected=False)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test_size = 0.2
train_data = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")
test_data = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")

features = ['date_time', 'deg_C', 'relative_humidity', 'absolute_humidity','sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
targets = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
# targets = ['target_carbon_monoxide', 'target_benzene']
# targets = ['target_nitrogen_oxides']

In [ ]:
deg_C = go.Scatter(
    x=train_data["date_time"],
    y=train_data["deg_C"]*10,
    name="deg_C",
#     line={
#         "color": "orange"
#     }
)

rel_humidity = go.Scatter(
    x=train_data["date_time"],
    y=train_data["relative_humidity"],
    name="relative_humidity",
#     line={
#         "color": "orange"
#     }
)

abs_humidity = go.Scatter(
    x=train_data["date_time"],
    y=train_data["absolute_humidity"]*40,
    name="absolute_humidity",
#     line={
#         "color": "orange"
#     }
)

sensor_1 = go.Scatter(
    x=train_data["date_time"],
    y=train_data["sensor_1"]/10,
    name="sensor_1",
#     line={
#         "color": "orange"
#     }
)

sensor_2 = go.Scatter(
    x=train_data["date_time"],
    y=train_data["sensor_2"]/10,
    name="sensor_2",
#     line={
#         "color": "orange"
#     }
)

sensor_3 = go.Scatter(
    x=train_data["date_time"],
    y=train_data["sensor_3"]/10,
    name="sensor_3",
#     line={
#         "color": "orange"
#     }
)

sensor_4 = go.Scatter(
    x=train_data["date_time"],
    y=train_data["sensor_4"]/10,
    name="sensor_4",
#     line={
#         "color": "orange"
#     }
)

sensor_5 = go.Scatter(
    x=train_data["date_time"],
    y=train_data["sensor_5"]/10,
    name="sensor_5",
#     line={
#         "color": "orange"
#     }
)

layout = {
    "title": "Prediction",
    "xaxis": {
        "rangeslider": {
            "visible": True
        },
        "type": "date"
    }
}

# offline.iplot({
#     "data": [deg_C, rel_humidity, abs_humidity, sensor_1, sensor_2, sensor_3, sensor_4, sensor_5],
#     "layout": layout
# })

In [ ]:
trace_test_data = go.Scatter(
    x=test_data["date_time"],
    y=test_data["deg_C"],
    name="deg_C Test",
    line={
        "color": "orange"
    }
)

# offline.iplot({
#     "data": [trace_test_data],
#     "layout": layout
# })

In [ ]:
train_sensor_failure = (train_data['date_time'] > "2010-04-08 21:00:00") & (train_data['date_time'] < "2010-04-10 00:00:00") | \
    (train_data['date_time'] > "2010-05-25 18:00:00") & (train_data['date_time'] < "2010-05-26 09:00:00") | \
    (train_data['date_time'] > "2010-06-19 12:00:00") & (train_data['date_time'] < "2010-06-21 05:00:00") | \
    (train_data['date_time'] > "2010-07-30 23:00:00") & (train_data['date_time'] < "2010-07-31 01:00:00") | \
    (train_data['date_time'] > "2010-08-09 15:00:00") & (train_data['date_time'] < "2010-08-09 20:00:00") | \
    (train_data['date_time'] > "2010-08-26 05:00:00") & (train_data['date_time'] < "2010-08-28 04:00:00") | \
    (train_data['date_time'] > "2010-09-07 21:00:00") & (train_data['date_time'] < "2010-09-08 08:00:00") | \
    (train_data['date_time'] > "2010-12-14 15:00:00") & (train_data['date_time'] < "2010-12-18 00:00:00") | \
    (train_data['date_time'] > "2010-12-24 18:00:00") & (train_data['date_time'] < "2010-12-25 00:00:00")
    
train_data = train_data[~train_sensor_failure]

In [ ]:
test_sensor_failure = (test_data['date_time'] > "2011-01-02 20:00:00") & (test_data['date_time'] < "2011-01-05 01:00:00") | \
    (test_data['date_time'] > "2011-01-28 16:00:00") & (test_data['date_time'] < "2011-01-29 02:00:00") | \
    (test_data['date_time'] > "2011-02-08 16:00:00") & (test_data['date_time'] < "2011-02-11 21:00:00")
    
test_data.loc[test_sensor_failure, [f for f in features if not f=="date_time"]] = test_data.mean(numeric_only=True).to_numpy()


In [ ]:
class DateTimeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        X["date_time"] = X["date_time"].apply(pd.to_datetime)
    
        day = X["date_time"].dt.dayofyear
        X["year_cos"] = np.cos((day-1)*(2*np.pi/365))
        X["year_sin"] = np.sin((day-1)*(2*np.pi/365))

        X["weekday"] = X["date_time"].dt.dayofweek.astype("category").isin([0,1,2,3,4]).astype("float64")

        hour = X["date_time"].dt.hour
        X["day_cos"] = np.cos((hour)*(2*np.pi/24))
        X["day_sin"] = np.sin((hour)*(2*np.pi/24))

        X = X.drop("date_time", axis=1)
    
        return X


class TargetTransformer(BaseEstimator, TransformerMixin):
    def fit(self, target):
        return self
    
    def transform(self, target):
        target = target.copy()
        return np.log1p(target)
    
    def inverse_transform(self, target):
        target = target.copy()
        return np.expm1(target)


In [ ]:
X = PolynomialFeatures(2).fit_transform(MinMaxScaler().fit_transform(DateTimeTransformer().transform(train_data[features])))
Y = TargetTransformer().transform(train_data[targets])

split_index = int(len(train_data)*(1-test_size))
x0_train = X[:split_index]
y0_train = Y[:split_index]
x0_test = X[split_index:]
y0_test = Y[split_index:]

# x0_train, x0_test, y0_train, y0_test = train_test_split(X, Y, test_size=0.2)


In [ ]:
reg_cm = MLPRegressor(hidden_layer_sizes=(100, 30, 5), activation="relu", max_iter=500)
# rs = RandomizedSearchCV(model, {
#     'hidden_layer_sizes': [(100,30,5), (50,100,20), (50,10)],
#     'activation': ['tanh', 'relu'],
#     'solver': ['sgd', 'adam'],
#     'alpha': [0.0001, 0.05],
#     'learning_rate': ['constant','adaptive'],
# })

# search = rs.fit(x_train, y_train["target_carbon_monoxide"])
# print(search.best_params_)


reg_cm.fit(X, Y["target_carbon_monoxide"])
print("\nMLP Regression")
print(reg_cm.score(X, Y["target_carbon_monoxide"]))
# print(reg_cm.score(x0_test, y0_test["target_carbon_monoxide"]))


In [ ]:
reg_bz = GradientBoostingRegressor()
# rs = RandomizedSearchCV(model, {
#     'loss': ['ls', 'lad'],
#     'n_estimators': [100, 1500],
#     'max_depth': [2, 15],
#     'min_samples_leaf': [1, 8],
#     'max_features': ['auto', 'sqrt', None]
# })

# search = rs.fit(x_train, y_train["target_benzene"])
# print(search.best_params_)

reg_bz.fit(X, Y["target_benzene"])

print("\nGradientBoosting Regression")
print(reg_bz.score(X, Y["target_benzene"]))
# print(reg_bz.score(x0_test, y0_test["target_benzene"]))



In [ ]:
reg_no = ElasticNetCV()
# rs = RandomizedSearchCV(model, {
#                'max_depth': [10, 100, None],
#                'max_features': ['auto', 'sqrt'],
#                'min_samples_leaf': [1, 2, 4],
#                'min_samples_split': [2, 5, 10],
#                'n_estimators': [130, 180, 230]})

# search = rs.fit(x_train, y_train["target_nitrogen_oxides"])
# print(search.best_params_)

reg_no.fit(X, Y["target_nitrogen_oxides"])

print("\nRandomForest Regression")
print(reg_no.score(X, Y["target_nitrogen_oxides"]))
# print(reg_no.score(x0_test, y0_test["target_nitrogen_oxides"]))

In [ ]:
X_test = X = PolynomialFeatures(2).fit_transform(MinMaxScaler().fit_transform(DateTimeTransformer().transform(test_data[features])))
y_submit = pd.DataFrame()
y_submit["date_time"] = test_data["date_time"]
y_submit["target_carbon_monoxide"] = TargetTransformer().inverse_transform(reg_cm.predict(X_test))
y_submit["target_benzene"] = TargetTransformer().inverse_transform(reg_bz.predict(X_test))
y_submit["target_nitrogen_oxides"] = TargetTransformer().inverse_transform(reg_no.predict(X_test))
y_submit.to_csv("./submission.csv", index=False)
y_submit.head()

In [ ]:
t_train = train_data.loc[:split_index, ["date_time"]].copy()
x_train = train_data.loc[:split_index, features].copy()
y_train = train_data.loc[:split_index, targets].copy()
t_test = train_data.loc[split_index:, ["date_time"]].copy()
x_test = train_data.loc[split_index:, features].copy()
y_test = train_data.loc[split_index:, targets].copy()


In [ ]:
# model_cm = TransformedTargetRegressor(regressor=Pipeline([
#     ("datetime_transformer", DateTimeTransformer()),
#     ("scaler", MinMaxScaler()),
#     ("poly", PolynomialFeatures(degree=2)),
#     ("model", reg_cm)
# ]), transformer=TargetTransformer(), check_inverse=False)

# model_bz = TransformedTargetRegressor(regressor=Pipeline([
#     ("datetime_transformer", DateTimeTransformer()),
#     ("scaler", MinMaxScaler()),
#     ("poly", PolynomialFeatures(degree=2)),
#     ("model", reg_bz)
# ]), transformer=TargetTransformer(), check_inverse=False)

# model_no = TransformedTargetRegressor(regressor=Pipeline([
#     ("datetime_transformer", DateTimeTransformer()),
#     ("scaler", MinMaxScaler()),
#     ("poly", PolynomialFeatures(degree=2)),
#     ("model", reg_no)
# ]), transformer=TargetTransformer(), check_inverse=False)


# model_cm.fit(x_train, y_train["target_carbon_monoxide"])
# print(model_cm.score(x_test, y_test["target_carbon_monoxide"]))

# model_bz.fit(x_train, y_train["target_benzene"])
# print(model_bz.score(x_test, y_test["target_benzene"]))

# model_no.fit(x_train, y_train["target_nitrogen_oxides"])
# print(model_no.score(x_test, y_test["target_nitrogen_oxides"]))

In [ ]:
# y_pred = pd.DataFrame({"target_carbon_monoxide_pred": reg_cm.predict(x_test), columns=map(lambda c: c + "_pred", targets), index=x_test.index)
pred = pd.concat([
    test_data["date_time"], 
#     "target_carbon_monoxide": y_test["target_carbon_monoxide"],
#     "target_carbon_monoxide_pred": y_pred["target_carbon_monoxide"],
#     "target_benzene": y_test["target_benzene"],
#     "target_benzene_pred": y_pred["target_benzene"],
    y_test,
    y_pred
], axis=1)


# carbon_monoxide = go.Scatter(
#     x=pred["date_time"],
#     y=pred["target_carbon_monoxide"],
#     name="target_carbon_monoxide",
#     line={
#         "color": "blue"
#     }
# )

# carbon_monoxide_pred = go.Scatter(
#     x=pred["date_time"],
#     y=pred["target_carbon_monoxide_pred"],
#     name="target_carbon_monoxide_pred",
#     line={
#         "color": "orange"
#     }
# )

# benzene = go.Scatter(
#     x=pred["date_time"],
#     y=pred["target_benzene"],
#     name="target_benzene",
#     line={
#         "color": "blue"
#     }
# )

# benzene_pred = go.Scatter(
#     x=pred["date_time"],
#     y=pred["target_benzene_pred"],
#     name="target_benzene_pred",
#     line={
#         "color": "orange"
#     }
# )

nitrogen_oxides = go.Scatter(
    x=pred["date_time"],
    y=pred["target_nitrogen_oxides"],
    name="target_nitrogen_oxides",
    line={
        "color": "blue"
    }
)

nitrogen_oxides_pred = go.Scatter(
    x=pred["date_time"],
    y=pred["target_nitrogen_oxides_pred"],
    name="target_nitrogen_oxides_pred",
    line={
        "color": "orange"
    }
)


layout = {
    "title": "Prediction",
    "xaxis": {
        "rangeslider": {
            "visible": True
        },
        "type": "date"
    }
}





# offline.iplot({
#     "data": [nitrogen_oxides, nitrogen_oxides_pred], 
#     "layout": layout
# })

In [ ]:
y_submit = pd.DataFrame(pipe.predict(test_data), columns=targets)
y_submit["date_time"] = test_data["date_time"]
y_submit.to_csv("./submission.csv", index=False)
y_submit.head()

In [ ]:
trace_y_pred = go.Scatter(
    x=y_submit["date_time"],
    y=y_submit["target_nitrogen_oxides"],
    name="Y pred",
    line={
        "color": "orange"
    }
)


offline.iplot({
    "data": [trace_y_pred],
    "layout": layout
})